# Preparing


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import os
import numpy as np
# !pip install array-to-latex

from os.path import exists
import pandas as pd

!pip install evaluate
!pip install rouge_score
!pip install openai==0.28
!pip install prv_accountant


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
  Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=71691f27dfb57387bb7b7ad5536eab7b02ebfc2a55f3a0e514b9b436f14ad1dc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Using cached prv_accountant-0.2.0-py3-none-any.whl (21 kB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

path = "/content/drive/Shareddrives/DP_ICL_Public" # path for all files

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Cosine Similarity


In [ ]:
import json
from sklearn.metrics.pairwise import cosine_similarity

label_path = path + "100_test_truth.json"
zero_shot_path = path + "gpt3da-0shot-100ensemble-100test2div.json"
four_shot_path = path + "gpt3da-4shot-100ensemble-100test2.json"

with open(label_path, 'r') as f:
  label = json.load(f)
with open(zero_shot_path, 'r') as f:
  zero_pred_f = json.load(f)
zero_pred = []
for i in range(len(zero_pred_f)):
  zero_pred.append(zero_pred_f[str(i)]['prediction'])
with open(four_shot_path, 'r') as f:
  four_pred_f = json.load(f)
four_pred = []
for i in range(len(four_pred_f)):
  four_pred.append(four_pred_f[str(i)]['prediction'])
print(label[0])
print(zero_pred[0])
print(four_pred[0])

Edson is booking his ticket now.

Joyce shared a link and Michael commented that it was a good deal, prompting Edson to quickly book a ticket.
 Joyce shared a link with Michael and Edson, who found it to be a good deal and decided to book a ticket.


In [ ]:
import evaluate
metric = evaluate.load("rouge")

def change_refer_pred(dr, en):
     data_references = []
     for data_refer in dr:
          for i in range(en):
               data_references.append(data_refer)
     return data_references
zero_pred_label = change_refer_pred(label, 100)
scores = metric.compute(predictions=zero_pred, references=zero_pred_label)
print("zero-shot score:", scores)


four_pred_label = change_refer_pred(label, 100)
scores = metric.compute(predictions=four_pred, references=four_pred_label)
print("four-shot score:", scores)


zero-shot score: {'rouge1': 0.34347216497969557, 'rouge2': 0.11688048788686325, 'rougeL': 0.26272165873735354, 'rougeLsum': 0.2627416056610946}
four-shot score: {'rouge1': 0.43330772548421037, 'rouge2': 0.1906731330309419, 'rougeL': 0.347897654907557, 'rougeLsum': 0.34787852237585815}


### Nonprivate

In [ ]:
embedding_four_shot = path + 'gpt3da-4shot-100ensemble-100test2eembedding.csv'
embedding_zero_shot = path + 'gpt3da-0shot-100ensemble-100test2divembeddingembedding.csv'

indexs = []
zero_shot_embedding  = pd.read_csv(embedding_zero_shot).values
four_shot_embedding  = pd.read_csv(embedding_four_shot).values
print(zero_shot_embedding.shape, four_shot_embedding.shape)

ensemble1 = 100
ensemble2 = 100

for i in range(100):
     # get mean embedding of df2
     curr_emb = four_shot_embedding[i*ensemble2:(i+1)*ensemble2]
     curr_emb_sum = np.sum(curr_emb, axis=0, keepdims=True)
     mean_emb = curr_emb_sum/ensemble1 # (1, 1536) 1 data, 1536 features
     dist = cosine_similarity(mean_emb, zero_shot_embedding[i*ensemble1:(i+1)*ensemble1])
     indexs.append(np.argmax(dist)+i*ensemble2)
 #print(indexs)
 # get predictions from json file
final_pred = []
for index in indexs:
     final_pred.append(zero_pred[index])

scores = metric.compute(predictions=final_pred, references=label)
print(" adj score", scores)





(10000, 1536) (10000, 1536)
 adj score {'rouge1': 0.4040835757939972, 'rouge2': 0.16506133308868248, 'rougeL': 0.3227834142771999, 'rougeLsum': 0.3235252374620169}


### private

In [ ]:
embedding_four_shot = path + 'gpt3da-4shot-100ensemble-100test2eembedding.csv'
embedding_zero_shot = path + 'gpt3da-0shot-100ensemble-100test2divembeddingembedding.csv'


zero_shot_embedding  = pd.read_csv(embedding_zero_shot).values
four_shot_embedding  = pd.read_csv(embedding_four_shot).values


print(zero_shot_embedding.shape, four_shot_embedding.shape)

from prv_accountant.dpsgd import find_noise_multiplier
size22 = 100
size11 = 100
num_step =100
for eps in [1,3,8]:
  sampling_probability = 100*4/14732 # 4 is 4-shot prediction, 14732 is the data size
  noise_multiplier = find_noise_multiplier(
                  sampling_probability=sampling_probability,
                  num_steps=num_step,
                  target_epsilon=eps,
                  target_delta=5e-5,
                  eps_error=0.01,
                  mu_max=100)
  print("noise", noise_multiplier)

  ensemble1 = 100
  ensemble2 = 100
  size1 = size11
  size2 = size22
  rouge1, rouge2, rougeL, rougeLsum = [],[],[],[]
  for _ in range(5):
    indexs = []
    for i in range(100):
        # get mean embedding of df2
        curr_emb = four_shot_embedding[(i*ensemble2):(i*ensemble2+size1)]
        curr_emb_sum = np.sum(curr_emb, axis=0, keepdims=True)
        curr_emb_sum += np.random.normal(loc=0, scale=noise_multiplier, size=curr_emb_sum.shape)
        mean_emb = curr_emb_sum/ensemble1 # (1, 1536) 1 data, 1536 features
        dist = cosine_similarity(mean_emb, zero_shot_embedding[(i*ensemble2):(i*ensemble2+size2)])
        indexs.append(np.argmax(dist)+i*ensemble2)

    final_pred = []
    for index in indexs:
        final_pred.append(zero_pred[index])

    score = metric.compute(predictions=final_pred, references=label)

    rouge1.append(score['rouge1'])
    rouge2.append(score['rouge2'])
    rougeL.append(score['rougeL'])
    rougeLsum.append(score['rougeLsum'])
  print("eps :", eps)
  print("rouge1's mean: ", np.mean(np.array(rouge1)), "rouge1's std: ", np.std(np.array(rouge1)))
  print("rouge2's mean: ", np.mean(np.array(rouge2)), "rouge2's std: ", np.std(np.array(rouge2)))
  print("rougeL's mean: ", np.mean(np.array(rougeL)), "rougeL's std: ", np.std(np.array(rougeL)))
  print("rougeLsum's mean: ", np.mean(np.array(rougeLsum)), "rougeLsum's std: ", np.std(np.array(rougeLsum)))


(10000, 1536) (10000, 1536)
noise 1.27186110977162
eps : 1
rouge1's mean:  0.3780967464645289 rouge1's std:  0.0030997026608231286
rouge2's mean:  0.14354214558656775 rouge2's std:  0.003739227247229012
rougeL's mean:  0.2950800721092415 rougeL's std:  0.005665949239475512
rougeLsum's mean:  0.29577568290356854 rougeLsum's std:  0.005875654934037727
noise 0.7870368587905289
eps : 3
rouge1's mean:  0.3912010755699068 rouge1's std:  0.005600481099570507
rouge2's mean:  0.15036945684834974 rouge2's std:  0.006627254751575122
rougeL's mean:  0.3072542844639169 rougeL's std:  0.004870206350579567
rougeLsum's mean:  0.3081601065364118 rougeLsum's std:  0.00486831399473413
noise 0.5481881634515281
eps : 8
rouge1's mean:  0.3973303902107222 rouge1's std:  0.00456113182905444
rouge2's mean:  0.15929606793622988 rouge2's std:  0.005081869790451159
rougeL's mean:  0.31348230027686186 rougeL's std:  0.003615679204142287
rougeLsum's mean:  0.3143403448678353 rougeLsum's std:  0.0037260658371122864


# Key Word Extraction

In [ ]:
import json
from sklearn.metrics.pairwise import cosine_similarity
label_path = path + "100_test_truth.json"
zero_shot_path = path + "gpt3da-0shot-test100.json"
four_shot_path = path + "gpt3da-4shot-100ensemble-100test2.json"

with open(label_path, 'r') as f:
  label = json.load(f)

with open(label_path, 'r') as f:
  label = json.load(f)
with open(zero_shot_path, 'r') as f:
  zero_pred_f = json.load(f)
zero_pred = []
for i in range(len(zero_pred_f)):
  zero_pred.append(zero_pred_f[str(i)]['prediction'])
with open(four_shot_path, 'r') as f:
  four_pred_f = json.load(f)
four_pred = []
for i in range(len(four_pred_f)):
  four_pred.append(four_pred_f[str(i)]['prediction'])

fourshot_pred = four_pred

import evaluate
metric = evaluate.load("rouge")

def change_refer_pred(dr, en):
     data_references = []
     for data_refer in dr:
          for i in range(en):
               data_references.append(data_refer)
     return data_references
zero_pred_label = change_refer_pred(label, 1)
scores = metric.compute(predictions=zero_pred, references=zero_pred_label)
print("zero-shot score:", scores)
scores = metric.compute(predictions=four_pred, references=four_pred_label)
print("four-shot score:", scores)





zero-shot score: {'rouge1': 0.3532397885072065, 'rouge2': 0.12692127348949506, 'rougeL': 0.2706758874386449, 'rougeLsum': 0.2708054072085293}
four-shot score: {'rouge1': 0.4332034565999593, 'rouge2': 0.19079464272450541, 'rougeL': 0.3477672056620673, 'rougeLsum': 0.34786986933427066}


## Key word without ranking

### Non-private KSA without ranking


In [ ]:
ds_size = 100 #test data size
repeat = 1
ensemble = 100
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
import openai
import evaluate
metric = evaluate.load("rouge")

stopword_set = set(stopwords.words('english'))

rouge1, rouge2, rougeL, rougeLsum = [],[],[],[]
for _ in range(repeat):

     # get predictions from json file
     final_pred = []
     count_threshold_list = []
     for i in tqdm(range(ds_size)):
          all_tokens = {} # key: token, value: count
          for j in range(ensemble):
               sentence = fourshot_pred[i*ensemble+j]
               tokens = nltk.word_tokenize(sentence)
               onegrams = set(ngrams(tokens, 1))
               # onegrams = set(onegrams)
               # making onegrams a set to avoid duplicate tokens
               for token in onegrams:
                    # only add one gram for one sentence
                    if token in all_tokens:
                         all_tokens[token] += 1
                    else:
                         all_tokens[token] = 1
          #print(all_tokens)
          all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
          #print(all_tokens_sorted)
          # ignore those non-words tokens
          filtered_tokens = {}
          for token, count in all_tokens_sorted:
               if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                    filtered_tokens[token] = count
          filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
          #print(filtered_tokens)
          # find the count threshold where the count gap is the largest
          actually_upper_bound = 0
          count_threshold = 0 # you need to creat a list to store all difference between the counts
          for k in range(len(filtered_tokens_sorted)-1):
               #print(k,len(filtered_tokens_sorted)-2)
               if filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1] > count_threshold:
                    count_threshold = filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1]
                    actually_upper_bound = filtered_tokens_sorted[k][1]
               if k == len(filtered_tokens_sorted)-2:
                    if  filtered_tokens_sorted[k+1][1] > count_threshold:
                         count_threshold = filtered_tokens_sorted[k+1][1]
                         actually_upper_bound =   filtered_tokens_sorted[k+1][1] #including all tokens

          #print(count_threshold)
          filtered_tokens = dict(filtered_tokens_sorted)
          filtered_tokens = [k[0] for k, v in filtered_tokens.items() if v >= actually_upper_bound]
          #print(filtered_tokens)

          random.shuffle(filtered_tokens) #shuffle the list of tokens
          prompt = '['
          for token in filtered_tokens:
               prompt += token + ', '
          prompt = prompt[:-2] + ']'
          prompt = prompt + '\nThe summary is:'
          zero_shot_sentence = zero_pred_f[str(i)]['origin_prompt'].replace('Summarize the above dialogue:','Summarize the above dialogue with the following word suggestions:')
          #print(zero_shot_sentence)
          final_prompt = zero_shot_sentence  + prompt
          # if i < 5 :
          #   print(final_prompt, label[i])

          ################################################################################
          ##### Please check your code and run this api to complete the experiment'#######
          ################################################################################

          openai.api_key = ""
          pred = openai.Completion.create(
               engine="text-davinci-003",
               prompt=final_prompt,
               temperature=0,
               max_tokens=256,
               top_p=1.0,
               frequency_penalty=0.0,
               presence_penalty=0.0
          )
          final_pred.append(pred['choices'][0]['text'])

     score = metric.compute(predictions=final_pred, references=label)
     rouge1.append(score['rouge1'])
     rouge2.append(score['rouge2'])
     rougeL.append(score['rougeL'])
     rougeLsum.append(score['rougeLsum'])
print("rouge1's mean: ", np.mean(np.array(rouge1)), "rouge1's std: ", np.std(np.array(rouge1)))
print("rouge2's mean: ", np.mean(np.array(rouge2)), "rouge2's std: ", np.std(np.array(rouge2)))
print("rougeL's mean: ", np.mean(np.array(rougeL)), "rougeL's std: ", np.std(np.array(rougeL)))
print("rougeLsum's mean: ", np.mean(np.array(rougeLsum)), "rougeLsum's std: ", np.std(np.array(rougeLsum)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/100 [00:00<?, ?it/s]


AuthenticationError: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.

### compute dp parameter

In [ ]:
!pip install autodp
from autodp.autodp_core import Mechanism
from autodp.transformer_zoo import Composition
from autodp import mechanism_zoo, transformer_zoo
from autodp import rdp_bank
from autodp import rdp_acct
from autodp import utils

import math

from autodp.privacy_calibrator import subsample_epsdelta

from scipy.optimize import minimize_scalar

import numpy as np
def rdp_to_approxdp(rdp, alpha_max=np.inf, BBGHS_conversion=True):
    # from RDP to approx DP
    # alpha_max is an optional input which sometimes helps avoid numerical issues
    # By default, we are using the RDP to approx-DP conversion due to BBGHS'19's Theorem 21
    # paper: https://arxiv.org/pdf/1905.09982.pdf
    # if you need to use the simpler RDP to approxDP conversion for some reason, turn the flag off

    def approxdp(delta):

        """
        approxdp outputs eps as a function of delta based on rdp calculations
        :param delta:
        :return: the \epsilon with a given delta
        """

        if delta < 0 or delta > 1:
            print("Error! delta is a probability and must be between 0 and 1")
        if delta == 0:
            return rdp(np.inf)
        else:
            def fun(x):  # the input the RDP's alpha
                if x <= 1:
                    return np.inf
                else:
                    if BBGHS_conversion:
                        return np.maximum(rdp(x) + np.log((x-1)/x) - (np.log(delta) + np.log(x))/(x-1), 0)
                    else:
                        return np.log(1 / delta) / (x - 1) + rdp(x)

            results = minimize_scalar(fun, method='Bounded', bracket=(1,2), bounds=(1, alpha_max) )
            if results.success:
                return results.fun
            else:
                # There are cases when certain \delta is not feasible.
                # For example, let p and q be uniform the privacy R.V. is either 0 or \infty and unless all \infty
                # events are taken cared of by \delta, \epsilon cannot be < \infty
                return np.inf
    return approxdp


def approxRDP_to_approxDP(delta, delta0, rdp_func, alpha_max=np.inf, BBGHS_conversion=True):

  if delta < delta0:
    return np.inf

  delta1 = delta - delta0

  approxdp = rdp_to_approxdp(rdp_func, alpha_max, BBGHS_conversion)

  return approxdp(delta1)



# eps is the privacy parameter for EM
# delta0 is the failure probability for PTR
class EMandPTR(Mechanism):
  def __init__(self, eps, delta0, sigma, name='PTR'):
    Mechanism.__init__(self)
    self.name=name
    self.params={'delta0':delta0, 'sigma':sigma}

    def privloss(t, alpha):
      return (np.exp(alpha*(eps-t)) - np.exp(-alpha*t) - (np.exp(alpha*eps-(alpha+1)*t) - np.exp(eps-(alpha+1)*t))) / ( np.exp(eps-t) - np.exp(-t) )



    def RDP_EM(alpha):
      temp = (np.sinh(alpha*eps) - np.sinh((alpha-1)*eps)) / np.sinh(eps)
      return min( 1/2 * alpha * eps**2, np.log(temp) / (alpha-1) )

    def approxRDP_PTR(alpha, delta0):
      return rdp_bank.RDP_gaussian({'sigma': sigma}, alpha)

    def approxRDP_EMandPTR(alpha, delta0):
      return rdp_bank.RDP_gaussian({'sigma': sigma}, alpha) + RDP_EM(alpha)

    def fakeRDP_EMandPTR(alpha):
      if alpha == np.inf:
        return np.inf
      return approxRDP_EMandPTR(alpha, delta0)

    self.propagate_updates(fakeRDP_EMandPTR, 'RDP')


class compose_subsampled_EMandPTR(Mechanism):
  def __init__(self, eps, delta0, sigma, prob, niter, name='compose_subsampled_EMandPTR'):

    Mechanism.__init__(self)
    self.name=name

    subsample = transformer_zoo.AmplificationBySampling() # by default this is using poisson sampling
    compose = transformer_zoo.Composition()

    mech = EMandPTR(eps, delta0, sigma)
    mech.neighboring = 'add_remove'

    if prob < 1:
      mech = subsample(mech, prob*(1-delta0) / (1-prob*delta0), improved_bound_flag=False)

    mech = compose([mech], [niter])
    rdp_total = mech.RenyiDP

    self.propagate_updates(rdp_total, type_of_update='RDP')


# params = {eps, delta0, sigma, prob, niter}
def compose_subsampled_EMandPTR_to_approxDP(params, delta):

  mech = compose_subsampled_EMandPTR(params['eps'], params['delta0'], params['sigma'], params['prob'], params['niter'])
  rdp_func = mech.RenyiDP

  return approxRDP_to_approxDP(delta, params['delta0']*params['prob'], rdp_func, alpha_max=200, BBGHS_conversion=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autodp: filename=autodp-0.2.3.1-py3-none-any.whl size=59816 sha256=79ee4895a47cc24928bc6bf61dde2bcfc3a56e31dce9e3f3fd26b516fc41c54c
  Stored in directory: /root/.cache/pip/wheels/bb/fa/09/a4bf92616f2b3353c47e75d36c14afc2aefce2b78ad6e3536f
Successfully built autodp


In [ ]:
delta = 5e-5
prob = 400/14732
# here you need to check and let the output of compose_subsampled_EMandPTR_to_approxDP
# to be less the your target epsilon


# privacy_params = {
#     'eps': 1,
#     'delta0': 0.99*delta/prob,
#     'sigma': 1.15046,
#     'prob': prob,
#     'niter': 100
# } #91

print(delta/prob)



privacy_params = {
    'eps': 0.9,
    'delta0': 0.99*delta/prob,
    'sigma': 1.869,
    'prob': prob,
    'niter': 100
} # 79

# privacy_params = {
#     'eps': 0.45,
#     'delta0': 0.8*delta/prob,
#     'sigma': 2.62,
#     'prob': prob,
#     'niter': 100
# } # 57


print(privacy_params['delta0'],privacy_params['eps'])
compose_subsampled_EMandPTR_to_approxDP(privacy_params, delta=delta)

0.0018415
0.0018230850000000001 0.9


2.9995872921472246

### private KSA with PTR

In [ ]:

ds_size = 100 #test data size
repeat = 1
ensemble = 100
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
import openai
import evaluate
metric = evaluate.load("rouge")
np.random.seed(123)
random.seed(123)
stopword_set = set(stopwords.words('english'))

count_pass = 0
gap_change = []
print(len(fourshot_pred))
rouge1, rouge2, rougeL, rougeLsum = [],[],[],[]
for _ in range(repeat):

     # get predictions from json file
     final_pred = []
     count_threshold_list = []
     for i in range(ds_size):
          all_tokens = {} # key: token, value: count
          for j in range(ensemble):
               sentence = fourshot_pred[i*ensemble+j]
               tokens = nltk.word_tokenize(sentence)
               onegrams = set(ngrams(tokens, 1))
               # onegrams = set(onegrams)
               # making onegrams a set to avoid duplicate tokens
               for token in onegrams:
                    # only add one gram for one sentence
                    if token in all_tokens:
                         all_tokens[token] += 1
                    else:
                         all_tokens[token] = 1
          #print(all_tokens)
          all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
          #print(all_tokens_sorted)
          # ignore those non-words tokens
          filtered_tokens = {}
          for token, count in all_tokens_sorted:
               if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                    filtered_tokens[token] = count
          filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
          #print(filtered_tokens)

          ### DP version (RNM)
          gap_lst = []
          for k in range( min(len(filtered_tokens_sorted)-1, 30) ):   # I assume we are only interested in k <= 30
            gap = filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1]
            gap_lst.append(gap)
            if k == len(filtered_tokens_sorted)-2:
              gap_lst.append(filtered_tokens_sorted[k+1][1])

          gap_lst = np.array(gap_lst)
          gap_max_prev = np.max(gap_lst)
          noisy_gap_lst = gap_lst + np.random.gumbel(0, 2/privacy_params['eps'], len(gap_lst))
          kstar = np.argmax(noisy_gap_lst)
          gap_max = np.max(noisy_gap_lst)
          gap_change.append(gap_max_prev-gap_max)


          #print(count_threshold)
          filtered_tokens = dict(filtered_tokens_sorted)
          print(filtered_tokens)

          ### Non-private
          #filtered_tokens = [k[0] for k, v in filtered_tokens.items() if v >= actually_upper_bound]

          ### DP version (PTR)
          from scipy import stats
          dk = gap_lst[kstar]
          noise1 = np.random.normal(0, 2*privacy_params['sigma'])
          noise2 = stats.norm.isf(privacy_params['delta0'], loc=0, scale=2*privacy_params['sigma'])
          dkhat = max(2, dk) + noise1 - noise2
          print('dk={}, noise1={}, noise2={}, dkhat={}'.format(dk, noise1, noise2, dkhat))
          if dkhat > 2:
            print("label", len(label[i]),"zero", len(zero_pred_f[str(i)]['prediction']),"four", len(fourshot_pred[i*ensemble+0]))
            print('***PASS TEST! RELEASE EXACT TOP-{} TOKENS***'.format(kstar))
            filtered_tokens = [k[0] for k, v in filtered_tokens.items()]
            filtered_tokens = filtered_tokens[:kstar]
            count_pass += 1
          else:
            print('***FAIL TEST! DO Zero-shot Learning***'.format(kstar))
            filtered_tokens = []
            print("label", len(label[i]),"zero", len(zero_pred_f[str(i)]['prediction']),"four", len(fourshot_pred[i*ensemble+0]))

          # print(filtered_tokens)
          if filtered_tokens == []:
            final_prompt = zero_pred_f[str(i)]['origin_prompt']
          else:
            random.shuffle(filtered_tokens) #shuffle the list of tokens
            prompt = '['
            for token in filtered_tokens:
                prompt += token + ', '
            prompt = prompt[:-2] + ']'
            prompt = prompt + '\nThe summary is:'
            zero_shot_sentence = zero_pred_f[str(i)]['origin_prompt'].replace('Summarize the above dialogue:','Summarize the above dialogue with the following word suggestions:')
            #print(zero_shot_sentence)
            final_prompt = zero_shot_sentence  + prompt
          if i < 5 :
            print(final_prompt)

          ################################################################################
          ##### Please check your code and run this api to complete the experiment'#######
          ################################################################################

          pred = openai.Completion.create(
               engine="text-davinci-003",
               prompt=final_prompt,
               temperature=0,
               max_tokens=256,
               top_p=1.0,
               frequency_penalty=0.0,
               presence_penalty=0.0
          )
          final_pred.append(pred['choices'][0]['text'])



     score = metric.compute(predictions=final_pred, references=label)
     rouge1.append(score['rouge1'])
     rouge2.append(score['rouge2'])
     rougeL.append(score['rougeL'])
     rougeLsum.append(score['rougeLsum'])
print("rouge1's mean: ", np.mean(np.array(rouge1)), "rouge1's std: ", np.std(np.array(rouge1)))
print("rouge2's mean: ", np.mean(np.array(rouge2)), "rouge2's std: ", np.std(np.array(rouge2)))
print("rougeL's mean: ", np.mean(np.array(rougeL)), "rougeL's std: ", np.std(np.array(rougeL)))
print("rougeLsum's mean: ", np.mean(np.array(rougeLsum)), "rougeLsum's std: ", np.std(np.array(rougeLsum)))

print(count_pass)
print(np.mean(np.array(gap_change)))


# ***PASS TEST! RELEASE EXACT TOP-3 TOKENS***
# rouge1's mean:  0.38549113959158926 rouge1's std:  0.004715646681797527
# rouge2's mean:  0.1442157014406559 rouge2's std:  0.005443081525002633
# rougeL's mean:  0.2958618078347388 rougeL's std:  0.0045467990552312655
# rougeLsum's mean:  0.29607762247992897 rougeLsum's std:  0.0043788640926342645


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


10000
{('Joyce',): 100, ('link',): 100, ('Edson',): 100, ('shared',): 96, ('Michael',): 93, ('ticket',): 93, ('cheap',): 50, ('good',): 48, ('deal',): 47, ('booking',): 46, ('book',): 42, ('found',): 41, ('decided',): 39, ('excited',): 24, ('away',): 19, ('right',): 19, ('commented',): 13, ('said',): 11, ('thought',): 10, ('think',): 6, ('They',): 5, ('showed',): 4, ('flight',): 3, ('find',): 3, ('surprised',): 3, ("'s",): 2, ('impressed',): 2, ('price',): 2, ('noticed',): 2, ('offer',): 1, ('deciding',): 1}
dk=43, noise1=-6.936372847080687, noise2=10.867315459438492, dkhat=25.196311693480816
label 32 zero 132 four 105
***PASS TEST! RELEASE EXACT TOP-5 TOKENS***

Dialogue:
 Joyce: Check this out!
Joyce: <link>
Michael: That's cheap!
Edson: No way! I'm booking my ticket now!!  
Summarize the above dialogue with the following word suggestions: [shared, link, Michael, Edson, Joyce]
The summary is:


AuthenticationError: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.

## Key word with ranking

In [ ]:
import json
from sklearn.metrics.pairwise import cosine_similarity
label_path = path + "100_test_truth.json"
zero_shot_path = path + "gpt3da-0shot-100ensemble-100test2div.json"


with open(zero_shot_path, 'r') as f:
  zero_pred_f2 = json.load(f)
zero_pred2 = []
for i in range(len(zero_pred_f2)):
  zero_pred2.append(zero_pred_f2[str(i)]['prediction'])


fourshot_pred = zero_pred2
print(len(fourshot_pred))

10000


### Non-private KSA with ranking

In [ ]:
ds_size = 100 #test data size
repeat = 1
ensemble = 100
kw_size = 10
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
import openai
import evaluate
metric = evaluate.load("rouge")

stopword_set = set(stopwords.words('english'))

rouge1, rouge2, rougeL, rougeLsum = [],[],[],[]
for _ in range(repeat):

     # get predictions from json file
     final_pred = []
     count_threshold_list = []
     for i in tqdm(range(ds_size)):
          all_tokens = {} # key: token, value: count
          for j in range(kw_size):
               sentence = fourshot_pred[i*ensemble+j]
               tokens = nltk.word_tokenize(sentence)
               onegrams = set(ngrams(tokens, 1))
               # onegrams = set(onegrams)
               # making onegrams a set to avoid duplicate tokens
               for token in onegrams:
                    # only add one gram for one sentence
                    if token in all_tokens:
                         all_tokens[token] += 1
                    else:
                         all_tokens[token] = 1
          #print(all_tokens)
          all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
          #print(all_tokens_sorted)
          # ignore those non-words tokens
          filtered_tokens = {}
          for token, count in all_tokens_sorted:
               if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                    filtered_tokens[token] = count
          filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
          #print(filtered_tokens)
          # find the count threshold where the count gap is the largest
          actually_upper_bound = 0
          count_threshold = 0 # you need to creat a list to store all difference between the counts
          for k in range(len(filtered_tokens_sorted)-1):
               #print(k,len(filtered_tokens_sorted)-2)
               if filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1] > count_threshold:
                    count_threshold = filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1]
                    actually_upper_bound = filtered_tokens_sorted[k][1]
               if k == len(filtered_tokens_sorted)-2:
                    if  filtered_tokens_sorted[k+1][1] > count_threshold:
                         count_threshold = filtered_tokens_sorted[k+1][1]
                         actually_upper_bound =   filtered_tokens_sorted[k+1][1] #including all tokens

          #print(count_threshold)
          filtered_tokens_new = {}
          for k, v in filtered_tokens.items():
            if v >= actually_upper_bound:
              if filtered_tokens_new.get(v) is None:
                filtered_tokens_new[v] = [k[0]]
              else:
                filtered_tokens_new[v].append(k[0])

          #print(filtered_tokens_new)
          for key in filtered_tokens_new:
              # Shuffle the elements within the key
              random.shuffle(filtered_tokens_new[key])
          #print(filtered_tokens_new)
          filtered_tokens = []
          for key in filtered_tokens_new:
              # Shuffle the elements within the key
              filtered_tokens+=filtered_tokens_new[key]
          print(filtered_tokens)
          prompt = '['
          for token in filtered_tokens:
               prompt += token + ', '
          prompt = prompt[:-2] + ']'
          prompt = prompt + '\nThe summary is:'
          zero_shot_sentence = zero_pred_f[str(i)]['origin_prompt'].replace('Summarize the above dialogue:','Summarize the above dialogue with the following word suggestions ranked by their frequency from high to low:')
          #print(zero_shot_sentence)
          final_prompt = zero_shot_sentence  + prompt
          if i < 5 :
            print(final_prompt, label[i])

          ################################################################################
          ##### Please check your code and run this api to complete the experiment'#######
          ################################################################################

          openai.api_key = ""
          pred = openai.Completion.create(
               engine="text-davinci-003",
               prompt=final_prompt,
               temperature=0,
               max_tokens=256,
               top_p=1.0,
               frequency_penalty=0.0,
               presence_penalty=0.0
          )
          final_pred.append(pred['choices'][0]['text'])

          # final_pred.append("test") # comment this to run.

          # print(final_pred[-1])


     score = metric.compute(predictions=final_pred, references=label)
     rouge1.append(score['rouge1'])
     rouge2.append(score['rouge2'])
     rougeL.append(score['rougeL'])
     rougeLsum.append(score['rougeLsum'])
print("rouge1's mean: ", np.mean(np.array(rouge1)), "rouge1's std: ", np.std(np.array(rouge1)))
print("rouge2's mean: ", np.mean(np.array(rouge2)), "rouge2's std: ", np.std(np.array(rouge2)))
print("rougeL's mean: ", np.mean(np.array(rougeL)), "rougeL's std: ", np.std(np.array(rougeL)))
print("rougeLsum's mean: ", np.mean(np.array(rougeLsum)), "rougeLsum's std: ", np.std(np.array(rougeLsum)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/100 [00:00<?, ?it/s]

['Edson', 'ticket', 'Joyce', 'book', 'link', 'Michael', 'decided']

Dialogue:
 Joyce: Check this out!
Joyce: <link>
Michael: That's cheap!
Edson: No way! I'm booking my ticket now!!  
Summarize the above dialogue with the following word suggestions ranked by their frequency from high to low: [Edson, ticket, Joyce, book, link, Michael, decided]
The summary is: Edson is booking his ticket now.


AuthenticationError: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.

### Exponential Mechanism

In [ ]:
!pip install autodp
from autodp.autodp_core import Mechanism
from autodp.transformer_zoo import Composition
from autodp import mechanism_zoo, transformer_zoo
from autodp import rdp_bank
from autodp import rdp_acct
from autodp import utils

import math

from autodp.privacy_calibrator import subsample_epsdelta

from scipy.optimize import minimize_scalar

import numpy as np

In [ ]:
def rdp_to_approxdp(rdp, alpha_max=np.inf, BBGHS_conversion=True):
    # from RDP to approx DP
    # alpha_max is an optional input which sometimes helps avoid numerical issues
    # By default, we are using the RDP to approx-DP conversion due to BBGHS'19's Theorem 21
    # paper: https://arxiv.org/pdf/1905.09982.pdf
    # if you need to use the simpler RDP to approxDP conversion for some reason, turn the flag off

    def approxdp(delta):

        """
        approxdp outputs eps as a function of delta based on rdp calculations
        :param delta:
        :return: the \epsilon with a given delta
        """

        if delta < 0 or delta > 1:
            print("Error! delta is a probability and must be between 0 and 1")
        if delta == 0:
            return rdp(np.inf)
        else:
            def fun(x):  # the input the RDP's alpha
                if x <= 1:
                    return np.inf
                else:
                    if BBGHS_conversion:
                        return np.maximum(rdp(x) + np.log((x-1)/x) - (np.log(delta) + np.log(x))/(x-1), 0)
                    else:
                        return np.log(1 / delta) / (x - 1) + rdp(x)

            results = np.min( [fun(alpha) for alpha in range(1, alpha_max)] )
            return results

            # results = minimize_scalar(fun, method='Bounded', bounds=(1, alpha_max) )
            # if results.success:
            #     return results.fun
            # else:
            #     # There are cases when certain \delta is not feasible.
            #     # For example, let p and q be uniform the privacy R.V. is either 0 or \infty and unless all \infty
            #     # events are taken cared of by \delta, \epsilon cannot be < \infty
            #     return np.inf
    return approxdp


def approxRDP_to_approxDP(delta, delta0, rdp_func, alpha_max=np.inf, BBGHS_conversion=True):

  if delta < delta0:
    return np.inf

  delta1 = delta - delta0

  approxdp = rdp_to_approxdp(rdp_func, alpha_max, BBGHS_conversion)

  return approxdp(delta1)

In [ ]:
# eps is the privacy parameter for EM
# add Gumbel(sensitivity/eps), where sensitivity is the sensitivity of utility function
class EM(Mechanism):
  def __init__(self, eps, name='EM', monotone=False):
    Mechanism.__init__(self)
    self.name=name
    self.params={'eps':eps}

    if monotone:

      def privloss(t, alpha):
        return (np.exp(alpha*(eps-t)) - np.exp(-alpha*t) - (np.exp(alpha*eps-(alpha+1)*t) - np.exp(eps-(alpha+1)*t))) / ( np.exp(eps-t) - np.exp(-t) )

      def RDP_EM(alpha):
        if alpha == np.infty:
          return eps
        enegt = ((alpha-1)*(np.exp(alpha*eps)-1)) / ((alpha)*(np.exp(alpha*eps)-np.exp(eps)))
        return np.log( privloss(np.log(1/enegt), alpha) ) / (alpha-1)

    else:

      def RDP_EM(alpha):
        if alpha == np.infty:
          return eps*2
        temp = (np.sinh(alpha*eps) - np.sinh((alpha-1)*eps)) / np.sinh(eps)
        return min( 1/2 * alpha * eps**2, np.log(temp) / (alpha-1) )

    self.propagate_updates(RDP_EM, 'RDP')


class compose_subsampled_EM(Mechanism):
  def __init__(self, eps, prob, niter, name='compose_subsampled_EM', monotone=False):

    Mechanism.__init__(self)
    self.name=name

    subsample = transformer_zoo.AmplificationBySampling() # by default this is using poisson sampling
    compose = transformer_zoo.Composition()

    mech = EM(eps, monotone=monotone)
    mech.neighboring = 'add_remove'

    if prob < 1:
      mech = subsample(mech, prob, improved_bound_flag=False)

    mech = compose([mech], [niter])
    rdp_total = mech.RenyiDP

    self.propagate_updates(rdp_total, type_of_update='RDP')


# params = {eps, prob, niter}
def compose_subsampled_EM_to_approxDP(params, delta):

  mech = compose_subsampled_EM(params['eps'], params['prob'], params['niter'], monotone=params['monotone'])
  rdp_func = mech.RenyiDP

  # for alpha in range(1, 200):
  #   print('rdp_func({})={}'.format(alpha, rdp_func(alpha)))

  approxdp = rdp_to_approxdp(rdp_func, alpha_max=200, BBGHS_conversion=True)

  return approxdp(delta)

In [ ]:
delta = 5e-5
prob = 400/14732

privacy_params = {
    'eps': 1.65,
    'sigma': 1,
    'prob': prob,
    'niter': 100,
    'monotone': False
}

compose_subsampled_EM_to_approxDP(privacy_params, delta=delta)

2.9984485729734365

In [ ]:
delta = 5e-5
prob = 400/14732

privacy_params = {
    'eps': 2.67,
    'sigma': 1,
    'prob': prob,
    'niter': 100,
    'monotone': False
}

compose_subsampled_EM_to_approxDP(privacy_params, delta=delta)

7.9957427730570085

### KSA-EM

In [ ]:
import json
from sklearn.metrics.pairwise import cosine_similarity
label_path = path + "100_test_truth.json"
zero_shot_path = path + "gpt3da-0shot-test100.json"
four_shot_path = path + "gpt3da-4shot-100ensemble-100test2.json"

with open(label_path, 'r') as f:
  label = json.load(f)
with open(zero_shot_path, 'r') as f:
  zero_pred_f = json.load(f)
zero_pred = []
for i in range(len(zero_pred_f)):
  zero_pred.append(zero_pred_f[str(i)]['prediction'])
with open(four_shot_path, 'r') as f:
  four_pred_f = json.load(f)
four_pred = []
for i in range(len(four_pred_f)):
  four_pred.append(four_pred_f[str(i)]['prediction'])
print(label[0])
print(zero_pred[0])
print(four_pred[0])
fourshot_pred = four_pred

Edson is booking his ticket now.

Joyce shared a link with Michael and Edson, and Edson was excited to find out that it was a good deal and decided to book a ticket.
 Joyce shared a link with Michael and Edson, who found it to be a good deal and decided to book a ticket.


In [ ]:
def make_diff_matrix(item_counts, k):
  """Makes diff matrix where diff_matrix[i,j] = c_i - c_j + uniquifying term.

  Args:
    item_counts: Array of item counts, sorted in decreasing order.
    k: Number of top counts desired.

  Returns:
    k x d matrix diff_matrix where diff_matrix[i,j] = c_i - c_j + (d(k-i-1) + j
    + 1) / (2dk) and c_1 <= c_2 <= ... <= c_d. diff_matrix is therefore strictly
    increasing along rows and strictly decreasing down columns. Note that the
    added uniquifying term is determined entirely by d, i, j, k and is therefore
    data-independent. Moreover, since the counts are integers, their differences
    are integers as well; the uniquifying terms, which have maximum
    dk / (2dk) = 0.5, therefore do not change the relative order of any
    non-identical count differences.
  """
  d = len(item_counts)
  base_along_row = np.arange(1, d + 1)
  base_down_col = np.arange(k - 1, -1, -1) * d
  uniquifying_terms = (base_along_row[np.newaxis, :] +
                       base_down_col[:, np.newaxis]) / (2 * d * k)
  return (item_counts[:k, np.newaxis] -
          item_counts[np.newaxis, :]) + uniquifying_terms

def get_diffs_to_positions(diff_matrix):
  """Computes array a where diff_matrix[a[0][i], a[1][i]] = sorted_diffs[i].

  Args:
    diff_matrix: Matrix of distinct count differences.

  Returns:
    Array a where diff_matrix[a[0][i], a[1][i]] = sorted_diffs[i], where
    sorted_diffs contains all entries of diff_matrix in increasing order.
  """
  # The below line of the code runs in time O(d * k * log(dk)). This could be
  # implemented more efficiently, leveraging the fact that diff_matrix is
  # strictly increasing along rows. This property allows us to use k-way merging
  # (https://en.wikipedia.org/wiki/K-way_merge_algorithm), which can bring the
  # runtime down to O(d * k * log(k)). However, since this function is not a
  # practical bottleneck in the code, we leave it as-is for now.
  return np.unravel_index(np.argsort(diff_matrix, axis=None), diff_matrix.shape)

def brute_compute_log_diff_counts(diff_matrix, sorted_diffs):
  """Computes array of log(# sequences w/ diff) for diff in diff_matrix.

  Args:
    diff_matrix: Matrix of distinct count differences.
    sorted_diffs: Diffs from diff_matrix, in increasing order.

  Returns:
    Array log_counts where, for array sorted_diffs of diffs from diff_matrix
    sorted in increasing order,
    log_counts[i] = log(# of sequences where largest count difference is diff
    from sorted_diffs[i]), computed using brute force.
  """
  k, d = diff_matrix.shape
  possible_sequences = itertools.permutations(np.arange(d), k)
  diffs_to_counts = np.zeros(d * k)
  for sequence in possible_sequences:
    diff = np.amax(
        [diff_matrix[row_idx, sequence[row_idx]] for row_idx in np.arange(k)])
    diff_idx = np.searchsorted(sorted_diffs, diff)
    diffs_to_counts[diff_idx] += 1
  # Ignore warnings from taking log(0). This produces -np.inf as intended.
  with np.errstate(divide="ignore"):
    return np.log(diffs_to_counts)

def compute_log_diff_counts(diff_matrix, diffs_to_positions):
  """Computes array of log(sequence count) for each diff in diff_matrix.

  Args:
    diff_matrix: Matrix of distinct count differences.
    diffs_to_positions: Dictionary mapping diffs to positions in diff_matrix.

  Returns:
    Array log_counts where, for array sorted_diffs of diffs from diff_matrix
    sorted in decreasing order, log_counts[i] = log(# of sequences where largest
    count difference is diff from sorted_diffs[i]), computed using Lemma 3.7
    (the definition of \tilde{m}) from the paper.

  Raises:
    RuntimeError: ns vector never filled.
  """
  k, d = diff_matrix.shape
  num_diffs = d * k
  log_diff_counts = np.empty(num_diffs)
  log_ns = np.empty(k)
  indices_filled = set()
  last_diff_idx_processed = -1
  # Ignore warnings from, respectively, taking logs of 0 or negative numbers.
  # log(0) becomes -np.inf as intended, and log(<0) becomes nan and is ignored.
  with np.errstate(divide="ignore"):
    with np.errstate(invalid="ignore"):
      updates = np.log((diffs_to_positions[1] + 1) - diffs_to_positions[0])
  for (diff_idx, i, u) in zip(range(num_diffs), diffs_to_positions[0], updates):
    if np.isnan(u):
      continue
    log_ns[i] = u
    indices_filled.add(i)
    if len(indices_filled) == k:
      last_diff_idx_processed = diff_idx
      break
  if last_diff_idx_processed == -1:
    raise RuntimeError("ns vector never filled")
  log_diff_counts[:last_diff_idx_processed] = -np.inf
  log_ns_sum = np.sum(log_ns)
  for (diff_idx, i, u) in zip(
      range(last_diff_idx_processed, num_diffs),
      diffs_to_positions[0][diff_idx:], updates[diff_idx:]):
    log_ns_sum -= log_ns[i]
    log_diff_counts[diff_idx] = log_ns_sum
    log_ns[i] = u
    log_ns_sum += log_ns[i]
  return log_diff_counts

def racing_sample(log_terms):
  """Numerically stable method for sampling from an exponential distribution.

  Args:
    log_terms: Array of terms of form log(coefficient) - (exponent term).

  Returns:
    A sample from the exponential distribution determined by terms. See
    Algorithm 1 from the paper "Duff: A Dataset-Distance-Based
    Utility Function Family for the Exponential Mechanism"
    (https://arxiv.org/pdf/2010.04235.pdf) for details; each element of terms is
    analogous to a single log(lambda(A_k)) - (eps * k/2) in their algorithm.

  Raises:
    RuntimeError: encountered inf or nan min time.
  """
  race_times = np.log(np.log(
      1.0 / np.random.uniform(size=log_terms.shape))) - log_terms
  winner = np.argmin(race_times)
  min_time = race_times[winner]
  if np.isnan(min_time) or np.isinf(min_time):
    raise RuntimeError(
        "Racing sample encountered inf or nan min time: {}".format(min_time))
  return winner

def sample_diff_idx(log_diff_counts, sorted_diffs, epsilon, sensitivity):
  """Racing samples a diff index from the exponential mechanism.

  Args:
    log_diff_counts: Array of log(# sequences with diff) for each diff in
      sorted_diffs.
    sorted_diffs: Increasing array of possible diffs.
    epsilon: Privacy parameter epsilon.
    neighbor_type: Available neighbor types are defined in the NeighborType
      enum.

  Returns:
    Index idx sampled from, for diff = sorted_diffs[idx], distribution
    P(diff) ~ count[diff] * exp(-epsilon * floor(diff) / 2).
  """
  return racing_sample(log_diff_counts - (epsilon * np.floor(sorted_diffs) /
                                          (2 * sensitivity)))

def sample_max_expo_distribution(expo_lambda, log_num_trials):
  """Computes max values from (simulated) draws from the expo distribution.

  Args:
    expo_lambda: Exponential distribution parameter; PDF of the distribution is
      p(x) = expo_lambda * exp(-expo_lambda * x).
    log_num_trials: Array where each entry is the log of the number of
      (simulated) draws from the exponential distribution to use in producing a
      max value.

  Returns:
    An array where entry i represents the max over exp(log_num_trials[i]) draws
    from the exponenttial distribution with parameter expo_lambda.

  Raises:
    RuntimeError: result contains inf or nan.
  """
  # Rather than actually sampling from the exponential distribution num_trials
  # times and taking the max (an O(num_trials) operation), we can simply draw
  # from a distribution that directly represents this max (an O(1) operation).
  # The probability that num_trials independent draws from a distribution are
  # all <= x is the CDF of that distribution raised to the n-th power.  Hence,
  # the CDF for the max that we need is the exponential distribution CDF raised
  # to the n-th power.  Inverting this CDF and plugging in a sample from the
  # uniform distribution on [0, 1] gives the desired max.
  num_results = len(log_num_trials)
  # The below line is more numerically stable than 1 / np.exp(log_num_trials).
  inverse_num_trials = np.exp(-log_num_trials)
  log_uniform_draws = np.log(np.random.uniform(size=num_results))
  results = -np.log(-np.expm1(
      np.multiply(inverse_num_trials, log_uniform_draws))) / expo_lambda
  if np.any(np.isnan(results)) or np.any(np.isinf(results)):
    raise RuntimeError(
        "Max expo sampler result contains inf or nan: {}".format(results))
  return results

def sample_diff_idx_via_pnf(log_diff_counts, sorted_diffs, epsilon,
                            sensitivity):
  """Samples an index of sorted_diffs according to permute-and-flip (PNF).

  Args:
    log_diff_counts: Array of log(# sequences with diff) for each diff in
      sorted_diffs.
    sorted_diffs: Increasing array of possible diffs.
    epsilon: Privacy parameter epsilon.
    neighbor_type: Available neighbor types are defined in the NeighborType
      enum.

  Returns:
    Index idx into sorted_diffs, sampled according to the permute-and-flip
    mechanism.

  Raises:
    RuntimeError: No noised value exceeded -infinity.
  """
  expo_lambda = epsilon / (2 * sensitivity)

  # Exclude entries with a count of zero.
  nonzero_count_indicator = ~np.isneginf(log_diff_counts)

  # Permute-and-flip is identical to report-noisy-max with exponential noise;
  # see the paper "The Permute-and-Flip Mechanism Is Identical to
  # Report-Noisy-Max with Exponential Noise"
  # (https://arxiv.org/pdf/2105.07260.pdf) for details.  The latter formulation
  # is simpler to implement efficiently, so that is what we use internally here.
  # Specifically, we draw a max noise value for each of the utility values (the
  # diffs), and add this max to the utility.  We then return the index where
  # this results in the largest noisy utility value.
  utilities = -np.floor(sorted_diffs[nonzero_count_indicator])
  noisy_utilities = utilities + sample_max_expo_distribution(
      expo_lambda, log_diff_counts[nonzero_count_indicator])
  if np.all(np.isneginf(noisy_utilities)):
    raise RuntimeError("No noised value exceeded -infinity")
  nonzero_count_indices = np.flatnonzero(nonzero_count_indicator)
  return nonzero_count_indices[np.argmax(noisy_utilities)]

def sequence_from_diff(diff,
                       diff_row,
                       diff_col,
                       diff_matrix,
                       sampler=lambda ary: np.random.choice(ary, 1)):
  """Samples a sequence with given diff uniformly at random.

  Args:
    diff: Diff (negative utility) of sequence to sample.
    diff_row: diff_matrix[diff_row, diff_col] = diff.
    diff_col: diff_matrix[diff_row, diff_col] = diff.
    diff_matrix: Matrix of distinct count differences.
    sampler: Function that selects an item from an array. Default value is
      uniform random choice.

  Returns:
    Array of item indices forming a sequence with utility -diff.
  """
  k = len(diff_matrix)
  sequence = np.full(k, diff_col)
  ts = [
      np.searchsorted(diff_matrix[row, :], diff, side="right")
      for row in range(k)
  ]
  for row in range(k):
    if row != diff_row:
      # The below line of the code runs in time O(dk), which technically makes
      # the runtime of the sequence_from_diff function O(dk^2). This could be
      # implemented more efficiently, bringing the runtime of the function down
      # to O(dk). However, this is not a practical bottleneck in the code, so we
      # leave it as-is for now.
      to_sample = [i for i in range(ts[row]) if i not in sequence]
      sequence[row] = sampler(to_sample)
  return sequence

def joint(item_counts,
          k,
          epsilon,
          neighbor_type,
          sample_diff_idx_func=sample_diff_idx):
  """Applies joint exponential mechanism to return sequence of top-k items.

  Args:
    item_counts: Array of item counts.
    k: Number of items with top counts to return.
    epsilon: Privacy parameter epsilon.
    neighbor_type: Available neighbor types are defined in the NeighborType
      enum.
    sample_diff_idx_func: Function to call for sampling a utility value.

  Returns:
    Array of k item indices as estimated by the joint exponential mechanism.
  """
  # Sort the counts in non-increasing order.
  sort_indices = np.argsort(item_counts)[::-1]
  item_counts = item_counts[sort_indices]

  # Note that the diff_matrix here is the negative of the \tilde{U} matrix from
  # the paper.
  diff_matrix = make_diff_matrix(item_counts, k)
  diffs_to_positions = get_diffs_to_positions(diff_matrix)
  log_diff_counts = compute_log_diff_counts(diff_matrix, diffs_to_positions)
  sorted_diffs = diff_matrix[diffs_to_positions]
  diff_idx = sample_diff_idx_func(log_diff_counts, sorted_diffs, epsilon,
                                  neighbor_type)
  diff_row, diff_col = diffs_to_positions[0][diff_idx], diffs_to_positions[1][
      diff_idx]
  sequence = sequence_from_diff(sorted_diffs[diff_idx], diff_row, diff_col,
                                diff_matrix)
  # Convert the indices returned by sequence_from_diff to the original item ids.
  # return sort_indices[sequence]
  return sequence

def pnf_joint(item_counts, k, epsilon, neighbor_type):
  """Applies joint permute-and-flip mechanism to return sequence of top-k items.

     Internals are identical to the exponential mechanism version of joint,
     except for the sample_diff_idx method.

  Args:
    item_counts: Array of item counts.
    k: Number of items with top counts to return.
    epsilon: Privacy parameter epsilon.
    neighbor_type: Available neighbor types are defined in the NeighborType
      enum.

  Returns:
    Array of k item indices as estimated by the joint permute-and-flip
    mechanism.
  """
  return joint(item_counts, k, epsilon, neighbor_type, sample_diff_idx_via_pnf)

In [ ]:
ds_size = 100 #test data size
repeat = 1
ensemble = 100
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
import openai
import time
import evaluate
metric = evaluate.load("rouge")

stopword_set = set(stopwords.words('english'))
# eps_1 = 0.77, eps_3 = 1.65, eps_8 = 2.67
rouge1, rouge2, rougeL, rougeLsum = [],[],[],[]
all_eps = [1.65]
for index, num_ensemble in enumerate([100]):
     eps =  all_eps[index]
     for _ in range(3):
          # get predictions from json file
          final_pred = []
          count_threshold_list = []
          for i in tqdm(range(ds_size)):
                # all_tokens = {} # key: token, value: count
                sentence = zero_pred_f[str(i)]["origin_prompt"]
                all_tokens = {}
                tokens = nltk.word_tokenize(sentence)
                onegrams = set(ngrams(tokens, 1))
                # onegrams = set(onegrams)
                # making onegrams a set to avoid duplicate tokens
                for token in onegrams:
                    # only add one gram for one sentence
                    if token in all_tokens:
                          all_tokens[token] += 0
                    else:
                          all_tokens[token] = 0
                for j in range(num_ensemble):
                      sentence = fourshot_pred[i*ensemble+j]
                      tokens = nltk.word_tokenize(sentence)
                      onegrams = set(ngrams(tokens, 1))
                      # onegrams = set(onegrams)
                      # making onegrams a set to avoid duplicate tokens
                      for token in onegrams:
                          # only add one gram for one sentence
                          if token in all_tokens:
                                all_tokens[token] += 1
                #print(all_tokens)
                all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
                #print(all_tokens_sorted)
                # ignore those non-words tokens
                filtered_tokens = {}
                for token, count in all_tokens_sorted:
                      if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                          filtered_tokens[token] = count
                filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
                item_counts = np.array([count for token, count in filtered_tokens_sorted])
                joint_out = joint(item_counts, k=min(10, len(item_counts)), epsilon=eps, neighbor_type=1)
                filtered_tokens = np.array(filtered_tokens_sorted, dtype=object)[joint_out]
                filtered_tokens = [token_tuple[0][0] for token_tuple in filtered_tokens]
                prompt = '['
                for token in filtered_tokens:
                      prompt += token + ', '
                prompt = prompt[:-2] + ']'
                prompt = prompt + '\nThe summary is:'
                zero_shot_sentence = zero_pred_f[str(i)]['origin_prompt'].replace('Summarize the above dialogue:','Summarize the above dialogue with the following word suggestions ranked by their frequency from high to low:')
                #print(zero_shot_sentence)
                final_prompt = zero_shot_sentence  + prompt
                # if i < 5 :
                #   print(final_prompt, label[i])

                ################################################################################
                ##### Please check your code and run this api to complete the experiment'#######
                ################################################################################

                openai.api_key = ""

                max_attempts = 5
                current_attempt = 1
                # pred = openai.Completion.create(
                #      engine="text-davinci-003",
                #      prompt=final_prompt,
                #      temperature=0,
                #      max_tokens=256,
                #      top_p=1.0,
                #      frequency_penalty=0.0,
                #      presence_penalty=0.0
                # )
                # final_pred.append(pred['choices'][0]['text'])

                while current_attempt <= max_attempts:
                    try:
                        pred = openai.Completion.create(
                            engine="text-davinci-003",
                            prompt=final_prompt,
                            temperature=0,
                            max_tokens=256,
                            top_p=1.0,
                            frequency_penalty=0.0,
                            presence_penalty=0.0
                        )
                        result = pred['choices'][0]['text']
                        final_pred.append(result)
                        break  # Exit the loop if the query is successful
                    except Exception as e:
                        print(f"Query failed with error: {e}")
                        current_attempt += 1  # Increment the attempt counter
                        if current_attempt <= max_attempts:
                            print("Retrying in 3 seconds...")
                            time.sleep(3)  # Wait for 3 seconds before the next query

            #final_pred.append("test") # comment this to run.

            # print(final_pred[-1])

          score = metric.compute(predictions=final_pred, references=label)
          rouge1.append(score['rouge1'])
          rouge2.append(score['rouge2'])
          rougeL.append(score['rougeL'])
          rougeLsum.append(score['rougeLsum'])
     print("rouge1's mean: ",    np.mean(np.array(rouge1)), "rouge1's std: ", np.std(np.array(rouge1)))
     print("rouge2's mean: ",    np.mean(np.array(rouge2)), "rouge2's std: ", np.std(np.array(rouge2)))
     print("rougeL's mean: ",    np.mean(np.array(rougeL)), "rougeL's std: ", np.std(np.array(rougeL)))
     print("rougeLsum's mean: ", np.array(rougeLsum), "rougeLsum's std: ", np.std(np.array(rougeLsum)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/100 [00:00<?, ?it/s]<ipython-input-38-7d27f7f7c2e5>:268: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sequence[row] = sampler(to_sample)


Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Retrying in 3 seconds...
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Retrying in 3 seconds...
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY),

  1%|          | 1/100 [00:12<20:38, 12.51s/it]

Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Retrying in 3 seconds...
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field

  2%|▏         | 2/100 [00:24<20:19, 12.44s/it]

...

Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Retrying in 3 seconds...
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field

  6%|▌         | 6/100 [01:14<19:28, 12.43s/it]

Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
Retrying in 3 seconds...
Query failed with error: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field

  6%|▌         | 6/100 [01:20<21:05, 13.47s/it]


KeyboardInterrupt: 